In [ ]:
# EPOCH: 3, threshold: 0.3563892316818238
# 0.8482403355136022 0.7547827810731125 0.7558116886139129
# (9008, 0.4295071047957371) (5974, 0.33093404753933714) (1420, 0.32535211267605635)


input_shape_num = (200, 24)

enc_num_tokens = None
INPUT_SIZE = 200 # 1-dimensional
CHANNELS = 24+4 # 2-dimensional



enc_nhead = 4 #кол-во multihead
DEPTH = 3 #кол-во энкодероа

enc_full_attention = False

DIM_K = 32 #main part of paper
DIM_D = 128 #размерность слоя head in multihead
DIM_FF = 128

enc_dropout_ff = 0.1
enc_dropout = 0.1

PARAMETER_SHARING = 'kv' #"layerwise" - базовый он вроде лучше скор имеет, но мб медленнее работает
activation = 'relu' # 'gelu'


input_num = tf.keras.layers.Input(shape=input_shape_num, name='input_numeric')
num_batchnorm = tf.keras.layers.LayerNormalization(axis=-2, epsilon=1e-6)(input_num)


ticker_size = df_1min['ticker_code'].max()+1 #18
input_emb = tf.keras.layers.Input(shape=(1,), name='input_ticker')
emb = tf.keras.layers.Embedding(ticker_size, 4) (input_emb)
emb_reshaped = tf.keras.layers.Reshape(target_shape=(4,))(emb)
emb_reshaped_repeat = tf.keras.layers.RepeatVector(input_shape_num[0])(emb_reshaped)



inputs = tf.keras.layers.concatenate([num_batchnorm, emb_reshaped_repeat])
#batch_norm = tf.keras.layers.BatchNormalization()(inputs) #Вот тут еще можно поиграться с LayerNorm

linformer = LinformerEnc(
            enc_num_tokens=enc_num_tokens,        # encoder setting
            enc_input_size=INPUT_SIZE,
            enc_channels=CHANNELS,
            enc_full_attention=enc_full_attention,
            enc_dim_k=DIM_K,
            enc_dim_d=DIM_D,
            enc_dim_ff=DIM_FF,
            enc_nhead = enc_nhead,
            enc_depth=DEPTH,
            enc_dropout_ff=enc_dropout_ff,
            enc_dropout=enc_dropout,
            enc_parameter_sharing=PARAMETER_SHARING,
            activation=activation)(inputs)



x = tf.keras.layers.GlobalAveragePooling1D()(linformer) #Почему бы не развернуть и не накинуть Dense слой?
#x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x) # хз насколько он здесь нужен, нужно поэксперементировать
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)


model = tf.keras.models.Model(inputs=[input_num, input_emb],\
                                       outputs=output)
model.summary()